In [4]:
!pip install sagemaker-experiments


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import sagemaker
import boto3
import pandas as pd
from datetime import datetime 

from sagemaker import image_uris 
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from time import gmtime, strftime
import time
from smexperiments import experiment

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = 'lawsnic-aiml-east2'
prefix = 'kaggle/customerChurn'

role = sagemaker.get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)


# name of experiment
timestep = datetime.now()
timestep = timestep.strftime("%d-%m-%Y-%H-%M-%S")
experiment_name = timestep + "xgboost-cust-churn"

# create experiment
experiment.Experiment.create(
    experiment_name=experiment_name,
    description="Iterative model tuning - altering data engineering steps",
    sagemaker_boto_client=sm,
)

Experiment(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7f30db3e5410>,experiment_name='18-02-2023-18-48-10xgboost-cust-churn',description='Iterative model tuning - altering data engineering steps',tags=None,experiment_arn='arn:aws:sagemaker:us-east-2:791580863750:experiment/18-02-2023-18-48-10xgboost-cust-churn',response_metadata={'RequestId': '2ac3036e-d866-4106-a727-61aed252e5db', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2ac3036e-d866-4106-a727-61aed252e5db', 'content-type': 'application/x-amz-json-1.1', 'content-length': '109', 'date': 'Sat, 18 Feb 2023 18:48:09 GMT'}, 'RetryAttempts': 0})

In [6]:
#sm.get_paginator('list_candidates_for_auto_ml_job')
#sm.list_candidates_for_auto_ml_job(AutoMLJobName='xgb-custChurn-2023-2-18-11')

In [7]:
# cell 10
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [8]:
# cell 11
#validation_location = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/test/CustomerChurnDW-2023-01-06T17-24-14/part-00000-3986a971-e8a5-4844-bfc8-e72e78f341b4-c000.csv'
#training_location = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train/CustomerChurnDW-2023-01-06T17-24-14/part-00000-0338ba51-05d0-4737-bd40-d52e24739eb6-c000.csv'

training_location = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train/CustomerChurnDW-2023-01-06T14-14-13/part-00000-642a0746-49b9-4fda-ad2d-98afe1db11ec-c000.csv'
validation_location = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/test/CustomerChurnDW-2023-01-06T14-14-13/part-00000-ccd6e4dd-898c-4fc4-a63a-85d1cfcfc4dc-c000.csv'

In [98]:
train_data = pd.read_csv(training_location)
train_data["SeniorCitizen"] = train_data["SeniorCitizen"].astype(int)
train_data["Churn"] = train_data["Churn"].astype(int)
train_data["Partner"] = train_data["Partner"].astype(int)
train_data["Dependents"] = train_data["Dependents"].astype(int)
train_data["PhoneService"] = train_data["PhoneService"].astype(int)
train_data["PaperlessBilling"] = train_data["PaperlessBilling"].astype(int)
train_data["isFemale"] = train_data["isFemale"].astype(int) 
train_data.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train_with_header.csv',index=False)
train_data.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train.csv',header=None,index=False)

print('train data has {} features'.format(len(train_data.columns)))

test_data = pd.read_csv(validation_location)
test_data["SeniorCitizen"] = test_data["SeniorCitizen"].astype(int)
test_data["Churn"] = test_data["Churn"].astype(int)
test_data["Partner"] = test_data["Partner"].astype(int)
test_data["Dependents"] = test_data["Dependents"].astype(int)
test_data["PhoneService"] = test_data["PhoneService"].astype(int)
test_data["PaperlessBilling"] = test_data["PaperlessBilling"].astype(int)
test_data["isFemale"] = test_data["isFemale"].astype(int)
test_data.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_target_and_header.csv',index=False)
test_data.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_target.csv',header=None,index=False)

print('validation data has {} features before dropping target'.format(len(test_data.columns)))

test_data2 =test_data.drop(["Churn"], axis=1) 
test_data2.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate.csv',header=None,index=False)
test_data2.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_header.csv',index=False)


s3_uri_train_data = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train.csv'
s3_uri_train_data_with_header = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train_with_header.csv'
s3_uri_validate_data = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate.csv'
s3_uri_validate_data_with_target_and_header = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_target_and_header.csv'
s3_uri_validate_data_with_target = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_target.csv'
s3_uri_validate_data_with_header = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_header.csv'

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=s3_uri_train_data, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=s3_uri_validate_data_with_target, content_type='csv')

train data has 41 features
validation data has 41 features before dropping target


First we'll need to specify training parameters to the estimator.  This includes:
1. The `xgboost` algorithm container
1. The IAM role to use
1. Training instance type and count
1. S3 location for output data
1. Algorithm hyperparameters

And then a `.fit()` function which specifies:
1. S3 location for output data.  In this case we have both a training and validation set which are passed in.

In [ ]:
from smexperiments.trial import Trial

trial = Trial.create(experiment_name=experiment_name, sagemaker_boto_client=sm)
experiment_config = {
    "ExperimentName": experiment_name,
    "TrialName": trial.trial_name,
    "TrialComponentDisplayName": "Data",
}

sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/manual-output/xgb-hpo'.format(bucket,prefix),
                                    sagemaker_session=sess,
                                    enable_sagemaker_metrics=True)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        eval_metric='auc',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation},
    experiment_config=experiment_config)

## HPO

In [11]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 2),
                            'max_depth': IntegerParameter(1, 10)}

objective_metric_name = 'validation:auc'

tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=20,
                            max_parallel_jobs=3)

In [12]:
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating hyperparameter tuning job with name: xgboost-230218-1851


............................................................................................!


In [13]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

'Completed'

In [14]:
boto3.client('sagemaker').describe_training_job(TrainingJobName=tuner.best_training_job())['FinalMetricDataList']

[{'MetricName': 'validation:auc',
  'Value': 0.819957971572876,
  'Timestamp': datetime.datetime(2023, 2, 18, 18, 59, 9, tzinfo=tzlocal())},
 {'MetricName': 'train:auc',
  'Value': 0.796550989151001,
  'Timestamp': datetime.datetime(2023, 2, 18, 18, 59, 9, tzinfo=tzlocal())},
 {'MetricName': 'ObjectiveMetric',
  'Value': 0.819957971572876,
  'Timestamp': datetime.datetime(2023, 2, 18, 18, 59, 9, tzinfo=tzlocal())}]

In [32]:
#modeltuner.best_training_job()
#boto3.client('sagemaker').describe_training_job(TrainingJobName=tuner.best_training_job())

s3_uri_model= boto3.client('sagemaker').describe_training_job(TrainingJobName=tuner.best_training_job())['ModelArtifacts']['S3ModelArtifacts']
print(s3_uri_model)

s3://lawsnic-aiml-east2/kaggle/customerChurn/manual-output/xgb-hpo/xgboost-230218-1851-019-a6ee6d9a/output/model.tar.gz


In [30]:
#xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
#predictor = tuner.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

model_name = 'xgbHPO-custChurn-model-' + str(int(time.time()))

In [35]:
primary_container = {"Image": container, "ModelDataUrl": s3_uri_model}

model = sm.create_model( 
    PrimaryContainer=primary_container,
    ModelName=model_name,
    ExecutionRoleArn=role
)

## Batch Inference

In [38]:
from time import gmtime, strftime, sleep 
from sagemaker.inputs import BatchDataCaptureConfig
from sagemaker.model_monitor import DataCaptureConfig
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

transform_job_name = 'xgbHPO-custchurn-batch-' + timestamp_suffix

transform_input = {
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': s3_uri_validate_data_with_header
                #'S3Uri': s3_uri_validate_data
            }
        },
        'ContentType': 'text/csv',
        'CompressionType': 'None',
        'SplitType': 'Line'
    }

transform_output = {
        'S3OutputPath': 's3://{}/{}/inference-results/xgbHPO/'.format(bucket, prefix),
        "Accept": "text/csv",
        'AssembleWith': 'Line',
    }

transform_resources = {
        'InstanceType': 'ml.m5.4xlarge',
        'InstanceCount': 1
    }

data_capture_config = {
       "DestinationS3Uri": 's3://{}/{}/model-monitor/batch-results/xgbHPO'.format(bucket,prefix), 
       "GenerateInferenceId": True,
    }

data_proc = { 
        "JoinSource": "Input"
        }
    
transformer = sm.create_transform_job(TransformJobName = transform_job_name,
                        ModelName = model_name,
                        TransformInput = transform_input,
                        TransformOutput = transform_output,
                        TransformResources = transform_resources,
                        DataCaptureConfig= data_capture_config,
                        DataProcessing = data_proc
)

In [39]:
print ('JobStatus')
print('----------')


describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
job_run_status = describe_response['TransformJobStatus']
print (job_run_status)

while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
    job_run_status = describe_response['TransformJobStatus']
    print (job_run_status)
    sleep(30)

JobStatus
----------
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
Completed


In [42]:
s3_output_key = '{}/inference-results/xgbHPO/validate.csv.out'.format(prefix);
local_inference_results_path = 'inference_results.csv'

s3 = boto3.resource('s3')
inference_results_bucket = s3.Bucket(bucket)

print(s3_output_key)

kaggle/customerChurn/inference-results/xgbHPO/validate.csv.out


In [44]:
inference_results_bucket.download_file(s3_output_key, local_inference_results_path);

data = pd.read_csv(local_inference_results_path, sep=',')
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

,0,0.1,0.2,10,0.3,0.4,29.75,301.9,1,0.0,...,0.0.16,1.0.8,0.0.17,0.0.18,1.0.9,0.0.19,0.0.20,0.08055959641933441,f830f3d7-d9fc-4014-ab56-074ca136f913,2023-02-18T19:31:10Z
0,0,0,0,16,1,0,18.95,326.80,0,1.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.033425,6d8e4a26-4e92-42f0-8894-c8c78ae71679,2023-02-18T19:31:10Z
1,0,1,1,10,1,0,55.20,528.35,1,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.017306,6e21a5ee-2e75-4398-98b3-17a55c8d1dbc,2023-02-18T19:31:10Z
2,0,0,1,17,1,1,64.70,1093.10,1,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.079891,e9c4fac4-fe1f-4273-8970-eeb75cf4251c,2023-02-18T19:31:10Z
3,1,1,0,71,1,1,96.35,6766.95,1,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.043323,284e2e62-8f4b-45b7-be7d-df115866da51,2023-02-18T19:31:10Z
4,1,1,1,60,1,1,74.85,4456.35,1,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.023446,b3c731a4-7cf4-4a49-ae28-e41ec4fff905,2023-02-18T19:31:10Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,0,0,0,26,0,0,35.75,1022.50,0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.120930,56fd25f1-2af0-494f-ba7b-3fa2dec75a67,2023-02-18T19:31:10Z
701,0,0,0,38,1,1,95.10,3691.20,0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.369698,824730cb-3166-4de7-85a2-14876375d962,2023-02-18T19:31:10Z
702,0,0,0,51,1,0,20.65,1020.75,1,1.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.033425,1c5bf318-63d2-4f65-95d9-f546145a6002,2023-02-18T19:31:10Z
703,0,1,1,1,1,1,70.65,70.65,0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.045573,b3589879-f447-4a5a-940e-c2f07d779d5d,2023-02-18T19:31:10Z


## Real time inference

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

import sys
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

predictionsT = xgb_predictor.predict(test_data.to_numpy()).decode('utf-8')

#print(predictionsT)

predictions = np.fromstring(predictionsT[1:], sep='\r\n')

## Async inference

In [ ]:
endpoint_config_name = resource_name.format(
    "EndpointConfig", datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.m5.xlarge",
            "InitialInstanceCount": 1,
        }
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            "S3OutputPath": f"s3://{s3_bucket}/{bucket_prefix}/output",
            # Optionally specify Amazon SNS topics
            # "NotificationConfig": {
            # "SuccessTopic": "arn:aws:sns:<aws-region>:<account-id>:<topic-name>",
            # "ErrorTopic": "arn:aws:sns:<aws-region>:<account-id>:<topic-name>",
            # }
        },
        "ClientConfig": {"MaxConcurrentInvocationsPerInstance": 4},
    },
)
print(f"Created EndpointConfig: {create_endpoint_config_response['EndpointConfigArn']}")

#####
endpoint_name = resource_name.format("Endpoint", datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

####
waiter = sm_client.get_waiter("endpoint_in_service")
print("Waiting for endpoint to create...")
waiter.wait(EndpointName=endpoint_name)
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(f"Endpoint Status: {resp['EndpointStatus']}")

In [ ]:
import os

def upload_file(input_location):
    prefix = f"{bucket_prefix}/input"
    return sm_session.upload_data(
        input_location,
        bucket=sm_session.default_bucket(),
        key_prefix=prefix,
        extra_args={"ContentType": "text/libsvm"},
    )


input_1_location = "validation.csv"
input_1_s3_location = upload_file(input_1_location)

response = sm_runtime.invoke_endpoint_async(
    EndpointName=endpoint_name, InputLocation=input_1_s3_location
)
output_location = response["OutputLocation"]
print(f"OutputLocation: {output_location}")

## Serverless real time inference

In [ ]:
xgboost_epc_name = "xgboost-serverless-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=xgboost_epc_name,
    ProductionVariants=[
        {
            "VariantName": "byoVariant",
            "ModelName": model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 4096,
                "MaxConcurrency": 1,
            },
        },
    ],
)

print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

endpoint_name = "xgboost-serverless-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=xgboost_epc_name,
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])